# Loop 4 LB Feedback Analysis

**LB Score**: 84.9010 (confirmed)
**CV Score**: 84.9010
**Gap**: 0.0000 (perfect alignment as expected for deterministic optimization)

## Key Observations
1. CV-LB alignment is perfect - this is a deterministic optimization problem
2. Current best: 84.90, Target: 68.93, Gap: 15.97 points (23.2%)
3. Current LB #1: 71.19 (terry_u16) - our target is BETTER than #1!
4. The crodoc kernel claims 74.75 but uses external datasets we don't have access to

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
import glob

getcontext().prec = 25
scale_factor = Decimal("1e18")

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def load_trees(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

def get_side(trees):
    if not trees:
        return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    return max(xys.max(axis=0) - xys.min(axis=0))

print("Functions loaded successfully")

Functions loaded successfully


In [2]:
# Analyze our current best solution
df = pd.read_csv('/home/code/experiments/004_ensemble_fixed/submission.csv')

# Calculate score per N
scores = []
for n in range(1, 201):
    trees = load_trees(n, df)
    if trees:
        side = get_side(trees)
        score = side**2 / n
        scores.append({'n': n, 'side': side, 'score': score})

scores_df = pd.DataFrame(scores)
print(f"Total score: {scores_df['score'].sum():.6f}")
print(f"\nWorst 20 N values by score contribution:")
print(scores_df.nlargest(20, 'score')[['n', 'side', 'score']].to_string())

Total score: 84.901044

Worst 20 N values by score contribution:
     n      side     score
0    1  0.813173  0.661250
18  19  3.057855  0.492130
48  49  4.900000  0.490000
20  21  3.200000  0.487619
30  31  3.869559  0.483016
19  20  3.101126  0.480849
49  50  4.900000  0.480200
36  37  4.200000  0.476757
24  25  3.436040  0.472255
52  53  5.000000  0.471698
25  26  3.500000  0.471154
50  51  4.900000  0.470784
33  34  4.000000  0.470588
31  32  3.878606  0.470112
75  76  5.950000  0.465822
21  22  3.200000  0.465455
37  38  4.200000  0.464211
53  54  5.000000  0.462963
51  52  4.900000  0.461731
27  28  3.590841  0.460505


In [3]:
# Calculate theoretical minimum (perfect packing)
# Tree area is approximately 0.35 * 1.0 = 0.35 (rough estimate)
# For N trees, minimum square side would be sqrt(N * tree_area)

tree_area = 0.35  # approximate
theoretical = []
for n in range(1, 201):
    min_side = np.sqrt(n * tree_area)
    min_score = min_side**2 / n
    theoretical.append({'n': n, 'theoretical_score': min_score})

theoretical_df = pd.DataFrame(theoretical)
print(f"Theoretical minimum total score: {theoretical_df['theoretical_score'].sum():.2f}")
print(f"\nOur score: {scores_df['score'].sum():.2f}")
print(f"Efficiency: {theoretical_df['theoretical_score'].sum() / scores_df['score'].sum() * 100:.1f}%")

Theoretical minimum total score: 70.00

Our score: 84.90
Efficiency: 82.4%


In [4]:
# Compare with leaderboard
print("Leaderboard Analysis:")
print("="*50)
print(f"#1 terry_u16: 71.19")
print(f"#2 c-number: 71.19")
print(f"#3 Rafbill: 71.26")
print(f"Our score: 84.90")
print(f"Target: 68.93")
print()
print(f"Gap to #1: {84.90 - 71.19:.2f} points ({(84.90 - 71.19)/71.19*100:.1f}%)")
print(f"Gap to target: {84.90 - 68.93:.2f} points ({(84.90 - 68.93)/68.93*100:.1f}%)")
print()
print("CRITICAL: Target 68.93 is BETTER than current #1 (71.19)!")
print("This requires techniques beyond what's publicly available.")

Leaderboard Analysis:
#1 terry_u16: 71.19
#2 c-number: 71.19
#3 Rafbill: 71.26
Our score: 84.90
Target: 68.93

Gap to #1: 13.71 points (19.3%)
Gap to target: 15.97 points (23.2%)

CRITICAL: Target 68.93 is BETTER than current #1 (71.19)!
This requires techniques beyond what's publicly available.


In [5]:
# Analyze where improvements are needed most
# Compare our scores to what would be needed to reach target

target = 68.93
current = 84.90
reduction_needed = (current - target) / current * 100

print(f"Need to reduce total score by {reduction_needed:.1f}%")
print(f"\nIf we improve uniformly, each N needs to improve by {reduction_needed:.1f}%")
print(f"\nAlternatively, focus on worst performers:")

# Calculate how much each N contributes to total
scores_df['pct_of_total'] = scores_df['score'] / scores_df['score'].sum() * 100
scores_df['cumulative_pct'] = scores_df.sort_values('score', ascending=False)['pct_of_total'].cumsum()

print("\nTop 20 worst N values contribute:")
worst_20 = scores_df.nlargest(20, 'score')
print(f"  {worst_20['pct_of_total'].sum():.1f}% of total score")
print(f"  If we halve these, we save {worst_20['score'].sum() * 0.5:.2f} points")

Need to reduce total score by 18.8%

If we improve uniformly, each N needs to improve by 18.8%

Alternatively, focus on worst performers:

Top 20 worst N values contribute:
  11.4% of total score
  If we halve these, we save 4.83 points


## Strategy Analysis

### Key Insights:
1. **Target is BETTER than LB #1** - This is extremely ambitious
2. **CV-LB alignment is perfect** - No distribution shift issues
3. **Gap is 15.97 points (23.2%)** - Need significant improvement

### What Top Competitors Likely Have:
1. **Better starting solutions** - Access to datasets with pre-computed solutions
2. **Longer optimization time** - 3+ hour runs vs our ~10 min
3. **bbox3 binary optimizer** - We're using C++ SA which may be less effective
4. **More sophisticated techniques** - Not shared publicly

### Recommended Strategy:
1. **Extended optimization** - Run C++ optimizer for 60+ minutes with higher params
2. **Target worst N values** - Focus on N=1, 19, 49, 21, 31 etc.
3. **Better backward propagation** - More aggressive tree dropping
4. **Fractional translation polish** - Very fine-grained adjustments
5. **Multiple optimization passes** - Iterate until no improvement